<a href="https://colab.research.google.com/github/vishalk31/Tensorflow-Specialization/blob/main/personalized_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing, utils


In [ ]:
conv_df = pd.read_csv("conv.csv",encoding="ISO-8859-1")

In [ ]:
conv_df.head()

,Unnamed: 0,chat
0,0,Thanks a lot... tgt some one is missing corre...
1,1,Mmm sry for late wishes. ..not well tats y
2,2,Ha ha no problem..cold huh??
3,3,Viral fever
4,4,Achoo take care of health


In [ ]:
questions, answers = [], []

for index, data in enumerate(conv_df.chat):
      if index < len(conv_df.chat)-1:
          questions.append(data)
          answers.append(conv_df.chat[index+1])

In [ ]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [ ]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [ ]:
import numpy as np
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)

In [ ]:
print(encoder_input_data.shape)

(87, 24)


In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

In [ ]:
print(encoder_input_data.shape)

In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

In [ ]:
print(decoder_output_data.shape)

(87, 24, 265)


In [ ]:
# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 24, 200)      53000       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 24, 200)      53000       ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=200) 

Epoch 1/200
3/3 [==============================] - 9s 158ms/step - loss: 1.3106
Epoch 2/200
3/3 [==============================] - 0s 151ms/step - loss: 1.2889
Epoch 3/200
3/3 [==============================] - 0s 151ms/step - loss: 1.2163
Epoch 4/200
3/3 [==============================] - 0s 153ms/step - loss: 1.1732
Epoch 5/200
3/3 [==============================] - 0s 147ms/step - loss: 1.1449
Epoch 6/200
3/3 [==============================] - 0s 151ms/step - loss: 1.1254
Epoch 7/200
3/3 [==============================] - 0s 151ms/step - loss: 1.1090
Epoch 8/200
3/3 [==============================] - 0s 148ms/step - loss: 1.0892
Epoch 9/200
3/3 [==============================] - 0s 156ms/step - loss: 1.0745
Epoch 10/200
3/3 [==============================] - 0s 150ms/step - loss: 1.0596
Epoch 11/200
3/3 [==============================] - 0s 156ms/step - loss: 1.0507
Epoch 12/200
3/3 [==============================] - 0s 150ms/step - loss: 1.0335
Epoch 13/200
3/3 [===================

In [ ]:
def inference():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = inference()